In [ ]:
import pyspark

# 1.Starting SparkSession

In [ ]:
# start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark_tutorial_part2").getOrCreate()

 # 2. Reading Big Data from Files

In [ ]:
# read the csv dataset
df = spark.read.csv("/content/loan_approval_dataset.csv", header=True, inferSchema=True)

In [ ]:
df.show()

+-------+-----------------+-------------+--------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+
|loan_id| no_of_dependents|    education| self_employed| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|
+-------+-----------------+-------------+--------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+
|      1|                2|     Graduate|            No|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|    Approved|
|      2|                0| Not Graduate|           Yes|      4100000|    12200000|         8|         417|                  2700000|                 220000

# 3. Inspecting and Understanding Data

In [ ]:
df.show(5)         # Shows top 5 rows
df.printSchema()   # Prints the data schema
df.columns         # Returns list of column names

+-------+-----------------+-------------+--------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+
|loan_id| no_of_dependents|    education| self_employed| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|
+-------+-----------------+-------------+--------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+
|      1|                2|     Graduate|            No|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|    Approved|
|      2|                0| Not Graduate|           Yes|      4100000|    12200000|         8|         417|                  2700000|                 220000

['loan_id',
 ' no_of_dependents',
 ' education',
 ' self_employed',
 ' income_annum',
 ' loan_amount',
 ' loan_term',
 ' cibil_score',
 ' residential_assets_value',
 ' commercial_assets_value',
 ' luxury_assets_value',
 ' bank_asset_value',
 ' loan_status']

In [ ]:
# # describe(): basic stats like count, mean, stddev, min, max
df.describe().show()

+-------+------------------+------------------+-------------+--------------+------------------+--------------------+------------------+------------------+-------------------------+------------------------+--------------------+-----------------+------------+
|summary|           loan_id|  no_of_dependents|    education| self_employed|      income_annum|         loan_amount|         loan_term|       cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|
+-------+------------------+------------------+-------------+--------------+------------------+--------------------+------------------+------------------+-------------------------+------------------------+--------------------+-----------------+------------+
|  count|              4269|              4269|         4269|          4269|              4269|                4269|              4269|              4269|                     4269|                    4269|                4269|

In [ ]:
# describe(): basic stats | summary(): extended stats with percentiles
df.summary().show()

+-------+------------------+------------------+-------------+--------------+------------------+--------------------+------------------+------------------+-------------------------+------------------------+--------------------+-----------------+------------+
|summary|           loan_id|  no_of_dependents|    education| self_employed|      income_annum|         loan_amount|         loan_term|       cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|
+-------+------------------+------------------+-------------+--------------+------------------+--------------------+------------------+------------------+-------------------------+------------------------+--------------------+-----------------+------------+
|  count|              4269|              4269|         4269|          4269|              4269|                4269|              4269|              4269|                     4269|                    4269|                4269|

# 4.Data Filtering & Selection

In [ ]:
# Remove leading/trailing spaces from all column names
df = df.toDF(*[c.strip() for c in df.columns])

In [ ]:
# Show only selected columns
df.select("loan_amount","loan_term").show()

# Filter rows where age > 30 using column expression
df.filter(df["loan_amount"] > 100000).show()

# Filter rows using SQL-style string expression
df.where("loan_term > 6 AND loan_amount > 100000").show()

+-----------+---------+
|loan_amount|loan_term|
+-----------+---------+
|   29900000|       12|
|   12200000|        8|
|   29700000|       20|
|   30700000|        8|
|   24200000|       20|
|   13500000|       10|
|   33000000|        4|
|   15000000|       20|
|    2200000|       20|
|    4300000|       10|
|   11200000|        2|
|   22700000|       18|
|   11600000|       16|
|   31500000|       14|
|    7400000|        6|
|   10700000|       10|
|    1600000|        4|
|    9400000|       14|
|   10300000|       10|
|   14600000|       12|
+-----------+---------+
only showing top 20 rows

+-------+----------------+-------------+-------------+------------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+
|loan_id|no_of_dependents|    education|self_employed|income_annum|loan_amount|loan_term|cibil_score|residential_assets_value|commercial_assets_value|luxury_assets_value|bank_asset_value|loan_status

# 5.Group By and Aggregate

In [ ]:
df.groupBy("self_employed").agg({"income_annum": "avg"}).show()

+-------------+-----------------+
|self_employed|avg(income_annum)|
+-------------+-----------------+
|          Yes|5065720.930232558|
|           No|5052430.391694196|
+-------------+-----------------+



# Better and more readable with functions:

In [ ]:
from pyspark.sql.functions import avg
df.groupBy("self_employed").agg(avg("income_annum").alias("avg_salary")).show()

+-------------+-----------------+
|self_employed|       avg_salary|
+-------------+-----------------+
|          Yes|5065720.930232558|
|           No|5052430.391694196|
+-------------+-----------------+



# 6.Modifying Columns

# Add a new column

In [ ]:
from pyspark.sql.functions import col

df = df.withColumn("income_annum", col("income"))

In [ ]:
df.show()

+-------+----------------+-------------+-------------+------------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+-------+
|loan_id|no_of_dependents|    education|self_employed|income_annum|loan_amount|loan_term|cibil_score|residential_assets_value|commercial_assets_value|luxury_assets_value|bank_asset_value|loan_status| income|
+-------+----------------+-------------+-------------+------------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+-------+
|      1|               2|     Graduate|           No|     9600000|   29900000|       12|        778|                 2400000|               17600000|           22700000|         8000000|   Approved|9600000|
|      2|               0| Not Graduate|          Yes|     4100000|   12200000|        8|        417|                 2700000|                2200000|            880000

# Rename a column

In [ ]:
df = df.withColumnRenamed("income_annum", "salary")

In [ ]:
df.show()

+-------+----------------+-------------+-------------+-------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+-------+
|loan_id|no_of_dependents|    education|self_employed| salary|loan_amount|loan_term|cibil_score|residential_assets_value|commercial_assets_value|luxury_assets_value|bank_asset_value|loan_status| income|
+-------+----------------+-------------+-------------+-------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+-------+
|      1|               2|     Graduate|           No|9600000|   29900000|       12|        778|                 2400000|               17600000|           22700000|         8000000|   Approved|9600000|
|      2|               0| Not Graduate|          Yes|4100000|   12200000|        8|        417|                 2700000|                2200000|            8800000|         3300000|   Rej

# Drop a column

In [ ]:
df = df.drop("income")

In [ ]:
df.show()

+-------+----------------+-------------+-------------+-------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+
|loan_id|no_of_dependents|    education|self_employed| salary|loan_amount|loan_term|cibil_score|residential_assets_value|commercial_assets_value|luxury_assets_value|bank_asset_value|loan_status|
+-------+----------------+-------------+-------------+-------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+
|      1|               2|     Graduate|           No|9600000|   29900000|       12|        778|                 2400000|               17600000|           22700000|         8000000|   Approved|
|      2|               0| Not Graduate|          Yes|4100000|   12200000|        8|        417|                 2700000|                2200000|            8800000|         3300000|   Rejected|
|      3|               3

# Join Operations (like SQL joins)
# Combining data from two DataFrames

In [ ]:
# Here,I am uploading 2nd dataset
df2 = spark.read.csv("/content/student_depression_dataset.csv", header=True, inferSchema=True)

In [ ]:
df.join(df2, df.loan_id == df2.id, "inner").show()

#"inner", "left", "right", "outer" are supported.

+-------+----------------+-------------+-------------+-------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+---+------+----+-------------+----------+-----------------+-------------+----+------------------+----------------+-------------------+--------------+----------+-------------------------------------+----------------+----------------+--------------------------------+----------+
|loan_id|no_of_dependents|    education|self_employed| salary|loan_amount|loan_term|cibil_score|residential_assets_value|commercial_assets_value|luxury_assets_value|bank_asset_value|loan_status| id|Gender| Age|         City|Profession|Academic Pressure|Work Pressure|CGPA|Study Satisfaction|Job Satisfaction|     Sleep Duration|Dietary Habits|    Degree|Have you ever had suicidal thoughts ?|Work/Study Hours|Financial Stress|Family History of Mental Illness|Depression|
+-------+----------------+-------------+-------------+----

In [ ]:
df.join(df2, df.loan_id == df2.id, "right").show()

+-------+----------------+-------------+-------------+-------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+---+------+----+-------------+----------+-----------------+-------------+----+------------------+----------------+-------------------+--------------+----------+-------------------------------------+----------------+----------------+--------------------------------+----------+
|loan_id|no_of_dependents|    education|self_employed| salary|loan_amount|loan_term|cibil_score|residential_assets_value|commercial_assets_value|luxury_assets_value|bank_asset_value|loan_status| id|Gender| Age|         City|Profession|Academic Pressure|Work Pressure|CGPA|Study Satisfaction|Job Satisfaction|     Sleep Duration|Dietary Habits|    Degree|Have you ever had suicidal thoughts ?|Work/Study Hours|Financial Stress|Family History of Mental Illness|Depression|
+-------+----------------+-------------+-------------+----

# 8. Write Output to File or Table

In [ ]:
# Save to CSV
df.write.csv("/content/loan_output_csv_v1", header=True)

In [ ]:
# Save to Parquet
df.write.parquet("/content/loan_output_parquet_v1")

In [ ]:
# Save to Hive table (requires Hive support)
df.write.mode("overwrite").saveAsTable("loan_approval_table_v1")

#  9.SQL Queries Using Spark SQL
# Convert a DataFrame to a SQL temporary view:

In [ ]:
df.createOrReplaceTempView("people")

In [ ]:
df.show()

+-------+----------------+-------------+-------------+-------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+
|loan_id|no_of_dependents|    education|self_employed| salary|loan_amount|loan_term|cibil_score|residential_assets_value|commercial_assets_value|luxury_assets_value|bank_asset_value|loan_status|
+-------+----------------+-------------+-------------+-------+-----------+---------+-----------+------------------------+-----------------------+-------------------+----------------+-----------+
|      1|               2|     Graduate|           No|9600000|   29900000|       12|        778|                 2400000|               17600000|           22700000|         8000000|   Approved|
|      2|               0| Not Graduate|          Yes|4100000|   12200000|        8|        417|                 2700000|                2200000|            8800000|         3300000|   Rejected|
|      3|               3

In [ ]:
#Run SQL queries:

result = spark.sql("SELECT salary FROM people WHERE salary > 100000")
result.show()

+-------+
| salary|
+-------+
|9600000|
|4100000|
|9100000|
|8200000|
|9800000|
|4800000|
|8700000|
|5700000|
| 800000|
|1100000|
|2900000|
|6700000|
|5000000|
|9100000|
|1900000|
|4700000|
| 500000|
|2900000|
|2700000|
|6300000|
+-------+
only showing top 20 rows



# 10. Working with RDDs (Low-level API)

# RDDs are the original Spark API (low-level). Not used as much anymore unless needed.

In [ ]:
rdd = spark.sparkContext.parallelize([1, 2, 3, 4])
squared = rdd.map(lambda x: x * x).collect()
print(squared)

#Use DataFrames instead when working with structured or semi-structured data — they are faster and easier to use.

[1, 4, 9, 16]
